In [1]:
D.<x> = PolynomialRing(QQ)
R = PolynomialRing(QQ,'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
Scplx= PowerSeriesRing(CC,'x',default_prec=300)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H4(n,m): # this is the definition in the published interpolations paper (2021):
    # def. 8.4
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    #if argument is an S type then 'polynomial' takes no argument.

def H4strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H4(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer


def H6(n,m):
    def base6(n,m):
        jay=J(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(jay-1))
        return L((-1)^m*(num/den)).O(n+1)
    return S(base6(n,m)^(1/(m-2))).polynomial()
     #if argument is an S type then 'polynomial' takes no argument.

def H6strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=H6(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return ((ddstep1(n,m)-ddstep2(n,m))/1728)

def deltaDaggerStrike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def deltaDagger2(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))

# def deltaDaggerStrike(n,m):
 #   return S(((H4strike(n,3)^3-H6strike(n,3)^2)/1728)).truncate(n+1)
    
def deltaDagger2Strike(n,m): #this applies the strike operator to the version of H4 that *is*
    #in the Integers paper.
    poly=deltaDagger2(n,m)
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [1..polydegree])
    return answer

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr



print("ok")

ok


In [2]:
data=[]
import time
start=time.time()/60
for m in [3..1203]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no6.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 100, 0.019011735916137695]
[4, 100, 0.031318336725234985]
[5, 100, 0.05441669747233391]
[6, 100, 0.06618649885058403]
[7, 100, 0.09174580127000809]
[8, 100, 0.11117013171315193]
[9, 100, 0.1393098644912243]
[10, 100, 0.15708009898662567]
[11, 100, 0.18494538217782974]
[12, 100, 0.20896315574645996]
[13, 100, 0.2383638173341751]
[14, 100, 0.25814251601696014]
[15, 100, 0.292423814535141]
[16, 100, 0.3156377337872982]
[17, 100, 0.34722984954714775]
[18, 100, 0.3695134334266186]
[19, 100, 0.4011284336447716]
[20, 100, 0.42913416773080826]
[21, 100, 0.4641644358634949]
[22, 100, 0.4853980988264084]
[23, 100, 0.5174749679863453]
[24, 100, 0.5463521033525467]
[25, 100, 0.5806393325328827]
[26, 100, 0.6042707301676273]
[27, 100, 0.6398145519196987]
[28, 100, 0.6693534515798092]
[29, 100, 0.7027332857251167]
[30, 100, 0.729024700820446]
[31, 100, 0.7623881325125694]
[32, 100, 0.7892531678080559]
[33, 100, 0.8276193216443062]
[34, 100, 0.8523523323237896]
[35, 100, 0.8913206160068512]
[36, 

[277, 100, 10.649531282484531]
[278, 100, 10.685558568686247]
[279, 100, 10.735291182994843]
[280, 100, 10.780800182372332]
[281, 100, 10.82718463614583]
[282, 100, 10.866682201623917]
[283, 100, 10.914438251405954]
[284, 100, 10.956762034446001]
[285, 100, 11.009624321013689]
[286, 100, 11.04700741916895]
[287, 100, 11.097268301993608]
[288, 100, 11.138699471950531]
[289, 100, 11.1887834854424]
[290, 100, 11.226952232420444]
[291, 100, 11.276346683502197]
[292, 100, 11.317195653915405]
[293, 100, 11.364632215350866]
[294, 100, 11.404721669852734]
[295, 100, 11.454018969088793]
[296, 100, 11.495111100375652]
[297, 100, 11.544810801744461]
[298, 100, 11.58014315366745]
[299, 100, 11.629277683794498]
[300, 100, 11.67525090277195]
[301, 100, 11.72508143633604]
[302, 100, 11.760144148021936]
[303, 100, 11.810365334153175]
[304, 100, 11.851285800337791]
[305, 100, 11.901371136307716]
[306, 100, 11.940328236669302]
[307, 100, 11.988262500613928]
[308, 100, 12.033357050269842]
[309, 100, 12.0

[545, 100, 23.152955785393715]
[546, 100, 23.197487734258175]
[547, 100, 23.24714033305645]
[548, 100, 23.2934254668653]
[549, 100, 23.34786443412304]
[550, 100, 23.39199885353446]
[551, 100, 23.44412548467517]
[552, 100, 23.492331869900227]
[553, 100, 23.544756468385458]
[554, 100, 23.58459347113967]
[555, 100, 23.641281351447105]
[556, 100, 23.68625070154667]
[557, 100, 23.73891231417656]
[558, 100, 23.781855169683695]
[559, 100, 23.835069451481104]
[560, 100, 23.883316688239574]
[561, 100, 23.939114466309547]
[562, 100, 23.978939585387707]
[563, 100, 24.03042323514819]
[564, 100, 24.07993458211422]
[565, 100, 24.13314500078559]
[566, 100, 24.174364387989044]
[567, 100, 24.230521999299526]
[568, 100, 24.276231449097395]
[569, 100, 24.32656253501773]
[570, 100, 24.372126266360283]
[571, 100, 24.42284446954727]
[572, 100, 24.470295302569866]
[573, 100, 24.524047501385212]
[574, 100, 24.56608498468995]
[575, 100, 24.62193451821804]
[576, 100, 24.667614117264748]
[577, 100, 24.7198044173

[815, 100, 36.66694771870971]
[816, 100, 36.71908028423786]
[817, 100, 36.776139568537474]
[818, 100, 36.82105369865894]
[819, 100, 36.881486002355814]
[820, 100, 36.93548446893692]
[821, 100, 36.98933116346598]
[822, 100, 37.03509321808815]
[823, 100, 37.090501453727484]
[824, 100, 37.139848470687866]
[825, 100, 37.200171899050474]
[826, 100, 37.24652573466301]
[827, 100, 37.302093517035246]
[828, 100, 37.35379120334983]
[829, 100, 37.40779427066445]
[830, 100, 37.452279664576054]
[831, 100, 37.50756576657295]
[832, 100, 37.55399828776717]
[833, 100, 37.61027343571186]
[834, 100, 37.65603758394718]
[835, 100, 37.71060395240784]
[836, 100, 37.76178828254342]
[837, 100, 37.818132635205984]
[838, 100, 37.861238952726126]
[839, 100, 37.91391909867525]
[840, 100, 37.968616016209126]
[841, 100, 38.02162730321288]
[842, 100, 38.063274167478085]
[843, 100, 38.11926793679595]
[844, 100, 38.16731943190098]
[845, 100, 38.22506918385625]
[846, 100, 38.27016654983163]
[847, 100, 38.32789005339146]

[1083, 100, 50.75669766962528]
[1084, 100, 50.80569798499346]
[1085, 100, 50.86441308632493]
[1086, 100, 50.90985931828618]
[1087, 100, 50.965161737054586]
[1088, 100, 51.01352873444557]
[1089, 100, 51.0733687505126]
[1090, 100, 51.12074061483145]
[1091, 100, 51.17595878243446]
[1092, 100, 51.23135429993272]
[1093, 100, 51.28658513352275]
[1094, 100, 51.33002520352602]
[1095, 100, 51.3897227011621]
[1096, 100, 51.438654500991106]
[1097, 100, 51.49405516684055]
[1098, 100, 51.5406144335866]
[1099, 100, 51.59841189906001]
[1100, 100, 51.652376268059015]
[1101, 100, 51.71034710109234]
[1102, 100, 51.7542981505394]
[1103, 100, 51.80933550000191]
[1104, 100, 51.86108082160354]
[1105, 100, 51.92044920474291]
[1106, 100, 51.96710964664817]
[1107, 100, 52.026134602725506]
[1108, 100, 52.07713770493865]
[1109, 100, 52.13307140022516]
[1110, 100, 52.182547301054]
[1111, 100, 52.23976005241275]
[1112, 100, 52.28984476625919]
[1113, 100, 52.35007146373391]
[1114, 100, 52.39513612166047]
[1115, 100

In [3]:
data=[]
import time
start=time.time()/60
for m in [1204..2403]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no7.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[1204, 100, 0.052285317331552505]
[1205, 100, 0.11233720183372498]
[1206, 100, 0.15990228205919266]
[1207, 100, 0.2184864655137062]
[1208, 100, 0.2691447511315346]
[1209, 100, 0.3308303505182266]
[1210, 100, 0.3792932517826557]
[1211, 100, 0.4375590533018112]
[1212, 100, 0.49157269671559334]
[1213, 100, 0.5478261709213257]
[1214, 100, 0.5934593677520752]
[1215, 100, 0.6554959490895271]
[1216, 100, 0.7048752680420876]
[1217, 100, 0.7605691663920879]
[1218, 100, 0.8110628686845303]
[1219, 100, 0.8668460994958878]
[1220, 100, 0.9202150516211987]
[1221, 100, 0.9803317524492741]
[1222, 100, 1.0251996368169785]
[1223, 100, 1.0818587355315685]
[1224, 100, 1.1363367699086666]
[1225, 100, 1.1982232332229614]
[1226, 100, 1.2429602146148682]
[1227, 100, 1.3009309843182564]
[1228, 100, 1.3519265688955784]
[1229, 100, 1.4078492373228073]
[1230, 100, 1.4583679512143135]
[1231, 100, 1.5137524865567684]
[1232, 100, 1.5675917826592922]
[1233, 100, 1.6268631182610989]
[1234, 100, 1.6736895181238651]
[12

[1465, 100, 14.343802466988564]
[1466, 100, 14.390978686511517]
[1467, 100, 14.450841251760721]
[1468, 100, 14.503653533756733]
[1469, 100, 14.561917152255774]
[1470, 100, 14.615166153758764]
[1471, 100, 14.67268680036068]
[1472, 100, 14.721143502742052]
[1473, 100, 14.780821517109871]
[1474, 100, 14.82798869907856]
[1475, 100, 14.88910960033536]
[1476, 100, 14.944296900182962]
[1477, 100, 15.003796435892582]
[1478, 100, 15.049407884478569]
[1479, 100, 15.110714234411716]
[1480, 100, 15.164616204798222]
[1481, 100, 15.222047880291939]
[1482, 100, 15.273220583796501]
[1483, 100, 15.331293750554323]
[1484, 100, 15.386350698769093]
[1485, 100, 15.451156634837389]
[1486, 100, 15.498273134231567]
[1487, 100, 15.556375235319138]
[1488, 100, 15.611037485301495]
[1489, 100, 15.667749732732773]
[1490, 100, 15.71575053408742]
[1491, 100, 15.778770487755537]
[1492, 100, 15.831666935235262]
[1493, 100, 15.89020086824894]
[1494, 100, 15.93902400508523]
[1495, 100, 16.00186026841402]
[1496, 100, 16.

[1724, 100, 28.812749303877354]
[1725, 100, 28.87768493592739]
[1726, 100, 28.925428837537766]
[1727, 100, 28.98584470152855]
[1728, 100, 29.038136400282383]
[1729, 100, 29.101066153496504]
[1730, 100, 29.15097488462925]
[1731, 100, 29.212225150316954]
[1732, 100, 29.266041550785303]
[1733, 100, 29.325079802423716]
[1734, 100, 29.376627434045076]
[1735, 100, 29.43844586983323]
[1736, 100, 29.49407035112381]
[1737, 100, 29.556100584566593]
[1738, 100, 29.606024581938982]
[1739, 100, 29.6657692193985]
[1740, 100, 29.72408440336585]
[1741, 100, 29.783754531294107]
[1742, 100, 29.832734767347574]
[1743, 100, 29.89670155569911]
[1744, 100, 29.947843220084906]
[1745, 100, 30.00975950062275]
[1746, 100, 30.05996898561716]
[1747, 100, 30.119893301278353]
[1748, 100, 30.174410417675972]
[1749, 100, 30.238091852515936]
[1750, 100, 30.28865122050047]
[1751, 100, 30.348548386245966]
[1752, 100, 30.402902234345675]
[1753, 100, 30.461621686816216]
[1754, 100, 30.50973742082715]
[1755, 100, 30.575048

[1987, 100, 43.82663080096245]
[1988, 100, 43.883657317608595]
[1989, 100, 43.94898158311844]
[1990, 100, 43.9998956695199]
[1991, 100, 44.06197755038738]
[1992, 100, 44.11918996647]
[1993, 100, 44.17910246551037]
[1994, 100, 44.2266854159534]
[1995, 100, 44.2947080694139]
[1996, 100, 44.3485562838614]
[1997, 100, 44.40862451866269]
[1998, 100, 44.46025017276406]
[1999, 100, 44.52113896608353]
[2000, 100, 44.576598569750786]
[2001, 100, 44.640364181250334]
[2002, 100, 44.69195293262601]
[2003, 100, 44.75064405426383]
[2004, 100, 44.80747511982918]
[2005, 100, 44.8690902851522]
[2006, 100, 44.91837561875582]
[2007, 100, 44.9807493686676]
[2008, 100, 45.03554840013385]
[2009, 100, 45.09790790081024]
[2010, 100, 45.15183075144887]
[2011, 100, 45.211514599621296]
[2012, 100, 45.2663279697299]
[2013, 100, 45.330444268882275]
[2014, 100, 45.37919723242521]
[2015, 100, 45.44306170195341]
[2016, 100, 45.49973903596401]
[2017, 100, 45.56024780496955]
[2018, 100, 45.609219796955585]
[2019, 100, 

[2250, 100, 59.11747178807855]
[2251, 100, 59.18018281459808]
[2252, 100, 59.23613565415144]
[2253, 100, 59.30109764635563]
[2254, 100, 59.35373493656516]
[2255, 100, 59.42102466896176]
[2256, 100, 59.478673316538334]
[2257, 100, 59.5413914360106]
[2258, 100, 59.592782970517874]
[2259, 100, 59.65956963598728]
[2260, 100, 59.719773065298796]
[2261, 100, 59.787017833441496]
[2262, 100, 59.84164686501026]
[2263, 100, 59.90564173460007]
[2264, 100, 59.96214013174176]
[2265, 100, 60.03101723641157]
[2266, 100, 60.083578035235405]
[2267, 100, 60.14782100170851]
[2268, 100, 60.210601683706045]
[2269, 100, 60.27442918717861]
[2270, 100, 60.3278130851686]
[2271, 100, 60.39394170045853]
[2272, 100, 60.44776338711381]
[2273, 100, 60.509049102663994]
[2274, 100, 60.56360587105155]
[2275, 100, 60.632419768720865]
[2276, 100, 60.69146800041199]
[2277, 100, 60.75923790037632]
[2278, 100, 60.811252132058144]
[2279, 100, 60.87422787025571]
[2280, 100, 60.937187902629375]
[2281, 100, 61.00041972100735]


In [2]:
data=[]
import time
start=time.time()/60
for m in [2404..3610]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no9.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[2404, 100, 0.057583048939704895]
[2405, 100, 0.12206731736660004]
[2406, 100, 0.17468643561005592]
[2407, 100, 0.23686020076274872]
[2408, 100, 0.2939843684434891]
[2409, 100, 0.35994651913642883]
[2410, 100, 0.412408284842968]
[2411, 100, 0.4761541858315468]
[2412, 100, 0.536301001906395]
[2413, 100, 0.600569985806942]
[2414, 100, 0.6511084884405136]
[2415, 100, 0.7211095169186592]
[2416, 100, 0.7779682353138924]
[2417, 100, 0.8418632857501507]
[2418, 100, 0.8977376706898212]
[2419, 100, 0.9612654186785221]
[2420, 100, 1.0227625519037247]
[2421, 100, 1.0888375006616116]
[2422, 100, 1.142305750399828]
[2423, 100, 1.2054380178451538]
[2424, 100, 1.2651423029601574]
[2425, 100, 1.330748837441206]
[2426, 100, 1.3819874003529549]
[2427, 100, 1.4481214880943298]
[2428, 100, 1.5054202154278755]
[2429, 100, 1.571230635046959]
[2430, 100, 1.6279001533985138]
[2431, 100, 1.6941123344004154]
[2432, 100, 1.7477945014834404]
[2433, 100, 1.8150046691298485]
[2434, 100, 1.8666271679103374]
[2435, 1

[2665, 100, 15.83124453574419]
[2666, 100, 15.881765604019165]
[2667, 100, 15.947864335030317]
[2668, 100, 16.00460458174348]
[2669, 100, 16.065972838550806]
[2670, 100, 16.121454935520887]
[2671, 100, 16.18180636689067]
[2672, 100, 16.2363738194108]
[2673, 100, 16.301008701324463]
[2674, 100, 16.353462252765894]
[2675, 100, 16.41707943379879]
[2676, 100, 16.475990667939186]
[2677, 100, 16.536921352148056]
[2678, 100, 16.588120851665735]
[2679, 100, 16.653887066990137]
[2680, 100, 16.71198983117938]
[2681, 100, 16.775743134319782]
[2682, 100, 16.829134203493595]
[2683, 100, 16.892338570207357]
[2684, 100, 16.95007411763072]
[2685, 100, 17.0170963332057]
[2686, 100, 17.068523600697517]
[2687, 100, 17.130451533943415]
[2688, 100, 17.18723126500845]
[2689, 100, 17.248375300318003]
[2690, 100, 17.301176887005568]
[2691, 100, 17.3670520670712]
[2692, 100, 17.423254001885653]
[2693, 100, 17.484671119600534]
[2694, 100, 17.536705002188683]
[2695, 100, 17.603458385914564]
[2696, 100, 17.658554

[2924, 100, 31.177067786455154]
[2925, 100, 31.245678052306175]
[2926, 100, 31.300815533846617]
[2927, 100, 31.361923284828663]
[2928, 100, 31.41985570266843]
[2929, 100, 31.48203395307064]
[2930, 100, 31.535295318812132]
[2931, 100, 31.600071419030428]
[2932, 100, 31.656494837254286]
[2933, 100, 31.7217849008739]
[2934, 100, 31.774711303412914]
[2935, 100, 31.84025977179408]
[2936, 100, 31.89605788514018]
[2937, 100, 31.96223260089755]
[2938, 100, 32.01350258663297]
[2939, 100, 32.07613863423467]
[2940, 100, 32.13985065370798]
[2941, 100, 32.202399134635925]
[2942, 100, 32.25374545156956]
[2943, 100, 32.31813285499811]
[2944, 100, 32.370876383036375]
[2945, 100, 32.43657151982188]
[2946, 100, 32.48865208774805]
[2947, 100, 32.55163539946079]
[2948, 100, 32.60852498561144]
[2949, 100, 32.6741137355566]
[2950, 100, 32.7267380990088]
[2951, 100, 32.79085498675704]
[2952, 100, 32.84958961978555]
[2953, 100, 32.912225637584925]
[2954, 100, 32.963601000607014]
[2955, 100, 33.03119185566902]

[3187, 100, 46.97956934943795]
[3188, 100, 47.035990349948406]
[3189, 100, 47.1016945540905]
[3190, 100, 47.156762667000294]
[3191, 100, 47.21896160021424]
[3192, 100, 47.28069945052266]
[3193, 100, 47.34354588389397]
[3194, 100, 47.39493430405855]
[3195, 100, 47.46303520351648]
[3196, 100, 47.52153253555298]
[3197, 100, 47.583582635968924]
[3198, 100, 47.63830640166998]
[3199, 100, 47.70316741988063]
[3200, 100, 47.75912259891629]
[3201, 100, 47.826564718037844]
[3202, 100, 47.87854568660259]
[3203, 100, 47.94220921769738]
[3204, 100, 48.00010196864605]
[3205, 100, 48.06600485369563]
[3206, 100, 48.11894318833947]
[3207, 100, 48.18463172018528]
[3208, 100, 48.240460488945246]
[3209, 100, 48.30268841981888]
[3210, 100, 48.35929337143898]
[3211, 100, 48.42481108382344]
[3212, 100, 48.48481595143676]
[3213, 100, 48.55285553634167]
[3214, 100, 48.604883566498756]
[3215, 100, 48.66910423338413]
[3216, 100, 48.72835476696491]
[3217, 100, 48.79130640625954]
[3218, 100, 48.84167406708002]
[32

[3450, 100, 62.86961745098233]
[3451, 100, 62.93518368527293]
[3452, 100, 62.99284600093961]
[3453, 100, 63.059073466807604]
[3454, 100, 63.111812233924866]
[3455, 100, 63.17884248495102]
[3456, 100, 63.23502951860428]
[3457, 100, 63.300126768648624]
[3458, 100, 63.35558916628361]
[3459, 100, 63.42216903343797]
[3460, 100, 63.481430884450674]
[3461, 100, 63.544829186052084]
[3462, 100, 63.5991881005466]
[3463, 100, 63.66283733397722]
[3464, 100, 63.721018966287374]
[3465, 100, 63.792394787073135]
[3466, 100, 63.845029801130295]
[3467, 100, 63.90882573649287]
[3468, 100, 63.97095053270459]
[3469, 100, 64.03424728289247]
[3470, 100, 64.08845899999142]
[3471, 100, 64.15449609607458]
[3472, 100, 64.21109085157514]
[3473, 100, 64.27500034868717]
[3474, 100, 64.3296850733459]
[3475, 100, 64.39574256911874]
[3476, 100, 64.45460933819413]
[3477, 100, 64.52018146589398]
[3478, 100, 64.57184168696404]
[3479, 100, 64.63597295060754]
[3480, 100, 64.69954008236527]
[3481, 100, 64.76182716712356]
[3

In [4]:
data=[]
import time
start=time.time()/60
for m in [3611..4818]:
    poly=H4(100,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/12sept22no10.txt','wb') # H4 series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3611, 100, 0.06645430624485016]
[3612, 100, 0.1310855709016323]
[3613, 100, 0.19622565433382988]
[3614, 100, 0.2505277320742607]
[3615, 100, 0.32101158797740936]
[3616, 100, 0.3783796355128288]
[3617, 100, 0.44484832137823105]
[3618, 100, 0.5001235865056515]
[3619, 100, 0.5680827833712101]
[3620, 100, 0.6284056529402733]
[3621, 100, 0.6969343349337578]
[3622, 100, 0.7491402365267277]
[3623, 100, 0.8132683038711548]
[3624, 100, 0.8736147321760654]
[3625, 100, 0.9412079490721226]
[3626, 100, 0.9973025321960449]
[3627, 100, 1.0657503195106983]
[3628, 100, 1.123953353613615]
[3629, 100, 1.1887482553720474]
[3630, 100, 1.2472237050533295]
[3631, 100, 1.3120440356433392]
[3632, 100, 1.3695291206240654]
[3633, 100, 1.4392768554389477]
[3634, 100, 1.4916533716022968]
[3635, 100, 1.5594743862748146]
[3636, 100, 1.623686883598566]
[3637, 100, 1.6900700852274895]
[3638, 100, 1.744690652936697]
[3639, 100, 1.8135329186916351]
[3640, 100, 1.876866102218628]
[3641, 100, 1.94306705519557]
[3642, 100

[3871, 100, 16.337689865380526]
[3872, 100, 16.394837886095047]
[3873, 100, 16.46095246449113]
[3874, 100, 16.514296885579824]
[3875, 100, 16.58037518709898]
[3876, 100, 16.643674451857805]
[3877, 100, 16.706451438367367]
[3878, 100, 16.76146014034748]
[3879, 100, 16.82803237065673]
[3880, 100, 16.88958388566971]
[3881, 100, 16.95376978442073]
[3882, 100, 17.00920147076249]
[3883, 100, 17.074567820876837]
[3884, 100, 17.13296226784587]
[3885, 100, 17.204718735069036]
[3886, 100, 17.258446589112282]
[3887, 100, 17.326202020049095]
[3888, 100, 17.387248903512955]
[3889, 100, 17.45316280424595]
[3890, 100, 17.50684031844139]
[3891, 100, 17.57473075389862]
[3892, 100, 17.635789405554533]
[3893, 100, 17.700829904526472]
[3894, 100, 17.757517755031586]
[3895, 100, 17.826252173632383]
[3896, 100, 17.88542353734374]
[3897, 100, 17.952634371817112]
[3898, 100, 18.006328098475933]
[3899, 100, 18.07261797040701]
[3900, 100, 18.13857515528798]
[3901, 100, 18.20300366729498]
[3902, 100, 18.25628432

[4130, 100, 32.491991970688105]
[4131, 100, 32.55992357060313]
[4132, 100, 32.620982218533754]
[4133, 100, 32.6859747171402]
[4134, 100, 32.74375203624368]
[4135, 100, 32.810073517262936]
[4136, 100, 32.870372001081705]
[4137, 100, 32.9398183375597]
[4138, 100, 32.99341280385852]
[4139, 100, 33.059806019067764]
[4140, 100, 33.12546616792679]
[4141, 100, 33.192030902951956]
[4142, 100, 33.245999187231064]
[4143, 100, 33.31437600031495]
[4144, 100, 33.3736192882061]
[4145, 100, 33.44065450131893]
[4146, 100, 33.49560375511646]
[4147, 100, 33.5624032728374]
[4148, 100, 33.623218171298504]
[4149, 100, 33.69141515344381]
[4150, 100, 33.74802194908261]
[4151, 100, 33.81553381681442]
[4152, 100, 33.87674067169428]
[4153, 100, 33.94109412282705]
[4154, 100, 33.99456823989749]
[4155, 100, 34.06482323631644]
[4156, 100, 34.12346035242081]
[4157, 100, 34.189471285790205]
[4158, 100, 34.24869788810611]
[4159, 100, 34.31411687284708]
[4160, 100, 34.37290807068348]
[4161, 100, 34.44123726710677]
[41

[4393, 100, 49.33199882134795]
[4394, 100, 49.3881498016417]
[4395, 100, 49.459729351103306]
[4396, 100, 49.52069940418005]
[4397, 100, 49.587106954306364]
[4398, 100, 49.64441345259547]
[4399, 100, 49.71283773705363]
[4400, 100, 49.77688816934824]
[4401, 100, 49.84740265458822]
[4402, 100, 49.90185236558318]
[4403, 100, 49.969567373394966]
[4404, 100, 50.031575452536345]
[4405, 100, 50.099559653550386]
[4406, 100, 50.1512825191021]
[4407, 100, 50.22062838450074]
[4408, 100, 50.28245495259762]
[4409, 100, 50.350133039057255]
[4410, 100, 50.411538023501635]
[4411, 100, 50.47976065054536]
[4412, 100, 50.539283752441406]
[4413, 100, 50.6073826700449]
[4414, 100, 50.66213405132294]
[4415, 100, 50.73056370019913]
[4416, 100, 50.79117250442505]
[4417, 100, 50.86010563373566]
[4418, 100, 50.91506129875779]
[4419, 100, 50.98340658470988]
[4420, 100, 51.04877212271094]
[4421, 100, 51.11532463878393]
[4422, 100, 51.17366560548544]
[4423, 100, 51.239211335778236]
[4424, 100, 51.30192243307829]
[4

[4657, 100, 66.50100810453296]
[4658, 100, 66.56016036868095]
[4659, 100, 66.63527963683009]
[4660, 100, 66.70064063742757]
[4661, 100, 66.77076283469796]
[4662, 100, 66.83208338543773]
[4663, 100, 66.90177068859339]
[4664, 100, 66.96599028632045]
[4665, 100, 67.04083511605859]
[4666, 100, 67.0978737026453]
[4667, 100, 67.16824625432491]
[4668, 100, 67.23560748621821]
[4669, 100, 67.30669478699565]
[4670, 100, 67.36655196920037]
[4671, 100, 67.43786823377013]
[4672, 100, 67.49861742183566]
[4673, 100, 67.56643759831786]
[4674, 100, 67.62692768871784]
[4675, 100, 67.70009023323655]
[4676, 100, 67.76475564017892]
[4677, 100, 67.83571480214596]
[4678, 100, 67.89130946993828]
[4679, 100, 67.96028103679419]
[4680, 100, 68.02810930088162]
[4681, 100, 68.09699783474207]
[4682, 100, 68.15277160704136]
[4683, 100, 68.2252810522914]
[4684, 100, 68.28742491826415]
[4685, 100, 68.35710295289755]
[4686, 100, 68.41793708503246]
[4687, 100, 68.48652182146907]
[4688, 100, 68.54826213419437]
[4689, 100